In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup    
import re
import pandas as pd
from pprint import pprint  
import numpy as np
import time 
from collections import Counter 
import json
import concurrent.futures

На сайте есть особенность, по любому запросу поиска показывается только 99 сраниц
Поэтому чтобы охватить больше объявлений сделаю градацию по ценам (от 10к до 250к, от 250 до 500к и т.д.) Тогда в этих ценовых рамках точно будет меньше 99 страниц с объявлениями.

In [4]:
# Создадим списки с марками нужных автомобилей и градацией цен
brands=['bmw','volkswagen','nissan','mercedes','toyota','audi','mitsubishi','skoda','volvo','honda','infiniti','lexus']
Grad_prices=['10000', '250000', '500000','750000', '1000000','1500000','2500000', '4000000','15000000']

In [9]:
# Используемы функции

def Count_pages(res):
    """Функция определения количества страниц с объявлениями"""
    pages=res.find_all('span',class_='Button__text')
    list_pages=re.findall('\d+',str(pages))
    if not list_pages:
        return 1
    else:
        max_page=max([int(item) for item in list_pages])
        return max_page

def Get_links(res):
    """Функция получения ссылок на объявления со страницы"""
    links=[]
    posts=res.find_all('a',class_='Link OfferThumb') 
    #print(posts)
    for link in posts:
        links.append(re.search('htt\w\S*/',str(link)).group(0))
    return links  

def Split_prices(brand):
    """Функция генерации url по диапазонам цен"""
    url=[]
    for i in range (1,len(Grad_prices)):
        url.append(str('https://auto.ru/cars/'+brand+'/used/?price_from='+Grad_prices[i-1]+'&price_to='+Grad_prices[i])) 
    return url

def fill_car_dict(ad):
    """Функция получения информации со страницы объявления"""
    car_dict={}
    car_url = ad
    time.sleep(np.random.rand()*2)
    response = requests.get(car_url)
    response.encoding = 'utf-8'
    page = BeautifulSoup(response.text, 'html.parser')
    try:
        json_data = json.loads(page.find('script', type="application/ld+json").string)
    except:
        json_data = None
    try:
        catalog_url = page.find('a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default').get('href')
    except:
        return None
    time.sleep(np.random.rand()*2)
    response_catalog = requests.get(catalog_url)
    response_catalog.encoding = 'utf-8'
    page_catalog = BeautifulSoup(response_catalog.text, 'html.parser')
    try:
        json_data_catalog = json.loads(page_catalog.find('script', type="application/json", id='initial-state').string)
    except:
        json_data_catalog = None
    try: json_data_equip = json.loads(page.find('script', type="application/json", id='initial-state').string)
    except: json_data_equip = None
    try:
        json_data['description'] = json_data['description'].replace('\n', ' ')
        json_data['description'] = re.sub('\W+',' ', json_data['description'])
    except:
        json_data['description'] = ''
    try:
        options_dict = json_data_catalog['state']['compare']['selected'][0]['options']
        options_list = [key for key in list(options_dict.keys()) if options_dict[key] == 1] 
    except:
        options_list = []
   
    try: car_dict['car_url'] = [json_data['offers']['url']]
    except: car_dict['car_url'] = [None]
    try: car_dict['bodyType'] = [json_data['bodyType']]
    except: car_dict['bodyType'] = [None]
    try: car_dict['brand'] = [json_data['brand']]
    except: car_dict['brand'] = [None]
    try: car_dict['color'] = [json_data['color']]
    except: car_dict['color'] = [None]
    try: car_dict['engineDisplacement'] = [json_data['vehicleEngine']['engineDisplacement']]
    except: car_dict['engineDisplacement'] = [None]
    try: car_dict['enginePower'] = [json_data['vehicleEngine']['enginePower']]
    except: car_dict['enginePower'] = [None]
    try: car_dict['fuelType'] = [json_data['fuelType']]
    except: car_dict['fuelType'] = [None]
    try: car_dict['image'] = [json_data['image']]
    except: car_dict['image'] = [None]
    try: car_dict['mileage'] = [page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[-1].text.replace(u'\xa0', u' ')]
    except: car_dict['mileage'] = [None]
    try: car_dict['modelDate'] = [json_data['modelDate']]
    except: car_dict['modelDate'] = [None]
    try: car_dict['model_name'] = [page.find_all('div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text]
    except: car_dict['model_name'] = [None]
    try: car_dict['numberOfDoors'] = [json_data['numberOfDoors']]
    except: car_dict['numberOfDoors'] = [None]
    try: car_dict['parsing_unixtime'] = [int(time.time())]
    except: car_dict['parsing_unixtime'] = [None]
    try: car_dict['price'] = [json_data['offers']['price']]
    except: car_dict['price'] = [None]
    try: car_dict['priceCurrency'] = [json_data['offers']['priceCurrency']]
    except: car_dict['priceCurrency'] = [None]
    try: car_dict['productionDate'] = [json_data['productionDate']]
    except: car_dict['productionDate'] = [None]
    try: car_dict['sell_id'] = [page.find('div', class_='CardHead__infoItem CardHead__id').text[2:]]
    except: car_dict['sell_id'] = [None]
    try: car_dict['views'] = [page.find('div', class_='CardHead__infoItem CardHead__views').text.split()[0]]
    except: car_dict['views'] = [None]
    try: car_dict['date_added'] = [page.find('div', class_='CardHead__infoItem CardHead__creationDate').text]
    except: car_dict['date_added'] = [None]
    try: car_dict['super_gen'] = [json.loads(page.find('div', id="sale-data-attributes").get('data-bem'))]
    except: car_dict['super_gen'] = [None]
    try: car_dict['vehicleConfiguration'] = [json_data['vehicleConfiguration']]
    except: car_dict['vehicleConfiguration'] = [None]
    try: car_dict['vehicleTransmission'] = [json_data['vehicleTransmission']]
    except: car_dict['vehicleTransmission'] = [None]
    try: car_dict['Владельцы'] = [page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all('span')[-1].text.replace(u'\xa0', u' ')]
    except: car_dict['Владельцы'] = [None]
    try: car_dict['ПТС'] = [page.find('li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text]
    except: car_dict['ПТС'] = [None]
    try: car_dict['Привод'] = [page.find('li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text]
    except: car_dict['Привод'] = [None]
    try: car_dict['Руль'] = [page.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text]
    except: car_dict['Руль'] = [None]
    try: car_dict['Состояние'] = [page.find('li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text]
    except: car_dict['Состояние'] = [None]
    try: car_dict['Таможня'] = [page.find('li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text]
    except: car_dict['Таможня'] = [None]
    return car_dict

In [11]:
dict_url={} # создадим словарь куда будем записывать сылки на страницы с объявлениями
for brand in brands:
    dict_url[brand]=Split_prices(brand) # заполним словарь по маркам автомобилей
dict_url  

{'bmw': ['https://auto.ru/cars/bmw/used/?price_from=10000&price_to=250000',
  'https://auto.ru/cars/bmw/used/?price_from=250000&price_to=500000',
  'https://auto.ru/cars/bmw/used/?price_from=500000&price_to=750000',
  'https://auto.ru/cars/bmw/used/?price_from=750000&price_to=1000000',
  'https://auto.ru/cars/bmw/used/?price_from=1000000&price_to=1500000',
  'https://auto.ru/cars/bmw/used/?price_from=1500000&price_to=2500000',
  'https://auto.ru/cars/bmw/used/?price_from=2500000&price_to=4000000',
  'https://auto.ru/cars/bmw/used/?price_from=4000000&price_to=15000000'],
 'volkswagen': ['https://auto.ru/cars/volkswagen/used/?price_from=10000&price_to=250000',
  'https://auto.ru/cars/volkswagen/used/?price_from=250000&price_to=500000',
  'https://auto.ru/cars/volkswagen/used/?price_from=500000&price_to=750000',
  'https://auto.ru/cars/volkswagen/used/?price_from=750000&price_to=1000000',
  'https://auto.ru/cars/volkswagen/used/?price_from=1000000&price_to=1500000',
  'https://auto.ru/car

Получаем ссылки на объявления

In [12]:
dict_cars={} # создадим словарь куда в итоге запишем ссылки на объявления по маркам автомобилей

for brand in dict_url.keys(): # перебираем марки автомобилей
    all_links=[] # в этот промежуточный список будем записывать ссылки на объявления 
  
    for price in range(len(Grad_prices)-1):
        # перебираем ценовых диапозонов
        
        print(brand,price)
        time.sleep(np.random.rand()*2) # ставим задержку перед очередным запросом
        response = BeautifulSoup(requests.get(dict_url[brand][price]).text,'html.parser') # получаем ответ от сервера
        empty = response.find_all('div',class_='ListingCarsFilters__submitButtonEmpty') # в случии пустой страницы вернет пустой список
    
        if not empty:
            # проверяем есть ли на странице объявления
            
            all_links+=Get_links(response)  # получаем ссылки на объявления с первой страницы
            count_pages = Count_pages(response) # смотрим сколько всего страниц с объявлениями
            print (count_pages)

            if count_pages>1:
                # если страниц больше одной
        
                for page in range (2,count_pages+1):
                    # перебираем страницы
                
                    time.sleep(np.random.rand()*2) # ставим задержку перед очередным запросом
                    try:
                        resp = BeautifulSoup(requests.get(dict_url[brand][price]+'&page='+str(page)).text,'html.parser') # отправляем запрос на следующую страницу
                    except:
                        print (dict_url[brand][price]+'&page='+str(page))
                    all_links+=Get_links(resp) # добавляем новые ссылки в промежуточный список
    
        else:
            time.sleep(np.random.rand()*2)
    dict_cars[brand] = all_links  # добавляем список сылок в словарь  

    
with open('cars.json', 'w', encoding='utf-8') as f:
    # запишем  в файл наши сылки на объявления, чтобы ничего не потерять
    json.dump(dict_cars, f, ensure_ascii=False, indent=4)  

bmw 0
16
bmw 1
29
bmw 2
37
bmw 3
35
bmw 4
61
bmw 5
78
bmw 6
58
bmw 7
40
volkswagen 0
56
volkswagen 1
61
volkswagen 2
87
https://auto.ru/cars/volkswagen/used/?price_from=500000&price_to=750000&page=55
https://auto.ru/cars/volkswagen/used/?price_from=500000&price_to=750000&page=57


KeyboardInterrupt: 

In [16]:
pprint(dict_cars)

{'bmw': ['https://auto.ru/cars/used/sale/bmw/3er/1105486347-779ddff4/',
         'https://auto.ru/cars/used/sale/bmw/5er/1103171666-91c45ea5/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105030068-122bf356/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105652473-3364332b/',
         'https://auto.ru/cars/used/sale/bmw/5er/1104777791-b21630d3/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105688943-63f2d4d1/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105211659-7f285937/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105648544-d355aba1/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105561887-f827ee91/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105689276-2db52a6f/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105545263-778fcbfb/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105679546-22aab564/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105689222-71d49028/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105689144-2fef

         'https://auto.ru/cars/used/sale/bmw/x5/1023117017-a315/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105661104-8badac0f/',
         'https://auto.ru/cars/used/sale/bmw/1er/1104768761-3c2b0f8d/',
         'https://auto.ru/cars/used/sale/bmw/x3/1105340322-ab8cf162/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105659054-a6313b5a/',
         'https://auto.ru/cars/used/sale/bmw/x5/1105660433-e303158a/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105660674-ac5c4b61/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105660431-00d937c9/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105177671-4c7a9e49/',
         'https://auto.ru/cars/used/sale/bmw/7er/1105660219-ac527d47/',
         'https://auto.ru/cars/used/sale/bmw/5er/1103384206-2c0883d7/',
         'https://auto.ru/cars/used/sale/bmw/7er/1102996223-ea0dea40/',
         'https://auto.ru/cars/used/sale/bmw/1er/1105158303-66415017/',
         'https://auto.ru/cars/used/sale/bmw/3er/1088080858-2f62066f/',

         'https://auto.ru/cars/used/sale/bmw/x5/1105136895-a15e04e6/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105296810-382cc14b/',
         'https://auto.ru/cars/used/sale/bmw/x5/1104795550-402755ea/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105634935-691204f4/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105572411-c0937159/',
         'https://auto.ru/cars/used/sale/bmw/x3/1105594547-8b569e2a/',
         'https://auto.ru/cars/used/sale/bmw/7er/1105368636-76db5640/',
         'https://auto.ru/cars/used/sale/bmw/x5/1087469512-5a5d6ef7/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105647202-463b9128/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105400563-2227b2b9/',
         'https://auto.ru/cars/used/sale/bmw/5er/1100970276-697e1451/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105408944-d3e676d8/',
         'https://auto.ru/cars/used/sale/bmw/1er/1102183261-1a993aaa/',
         'https://auto.ru/cars/used/sale/bmw/6er/1105577082-e35d7d61

         'https://auto.ru/cars/used/sale/bmw/3er/1105223970-cfc5665a/',
         'https://auto.ru/cars/used/sale/bmw/x5/1104713570-b1e8aec1/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105599254-b8e117d5/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105482601-8aef11d5/',
         'https://auto.ru/cars/used/sale/bmw/5er/1079619784-d93eaab3/',
         'https://auto.ru/cars/used/sale/bmw/x3/1105632101-008d586e/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105078874-bfcbafda/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105637822-16a71b7c/',
         'https://auto.ru/cars/used/sale/bmw/x3/1105644264-f47d3973/',
         'https://auto.ru/cars/used/sale/bmw/5er/1104820406-4fb28011/',
         'https://auto.ru/cars/used/sale/bmw/3er/1104825943-f64c5919/',
         'https://auto.ru/cars/used/sale/bmw/x1/1105638892-937df382/',
         'https://auto.ru/cars/used/sale/bmw/x5/1105620254-8bbae02b/',
         'https://auto.ru/cars/used/sale/bmw/7er/1105217588-21256293/

         'https://auto.ru/cars/used/sale/bmw/x6/1105309058-14530ee6/',
         'https://auto.ru/cars/used/sale/bmw/5er/1101594065-b07140b3/',
         'https://auto.ru/cars/used/sale/bmw/5er/1086672618-3d28b1a1/',
         'https://auto.ru/cars/used/sale/bmw/x5/1104722517-ccdf01df/',
         'https://auto.ru/cars/used/sale/bmw/5er/1103937306-3f842fab/',
         'https://auto.ru/cars/used/sale/bmw/x5/1105087015-928d6f47/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105494232-873680df/',
         'https://auto.ru/cars/used/sale/bmw/x4/1103349505-dae092c6/',
         'https://auto.ru/cars/used/sale/bmw/x5/1084151543-f68f7d3e/',
         'https://auto.ru/cars/used/sale/bmw/4/1104869514-13c62df5/',
         'https://auto.ru/cars/used/sale/bmw/x5/1104827916-4036e998/',
         'https://auto.ru/cars/used/sale/bmw/x6_m/1082121282-d6d60ce0/',
         'https://auto.ru/cars/used/sale/bmw/7er/1089078442-ddde0ec6/',
         'https://auto.ru/cars/used/sale/bmw/m3/1100226028-15778863/',


         'https://auto.ru/cars/used/sale/bmw/x1/1105180611-d780b7ad/',
         'https://auto.ru/cars/used/sale/bmw/x6/1103756594-796c1a40/',
         'https://auto.ru/cars/used/sale/bmw/6er/1104650385-720cf3b5/',
         'https://auto.ru/cars/used/sale/bmw/x1/1105303663-3bf14fc8/',
         'https://auto.ru/cars/used/sale/bmw/x3/1105383292-e5ef5f18/',
         'https://auto.ru/cars/used/sale/bmw/x4/1105303670-eb42b95d/',
         'https://auto.ru/cars/used/sale/bmw/x5/1105311899-7f201ff1/',
         'https://auto.ru/cars/used/sale/bmw/x5/1105616214-bcedf6d0/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105446467-7ccd6894/',
         'https://auto.ru/cars/used/sale/bmw/7er/1104981969-a098c206/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105556508-f1ca6dc9/',
         'https://auto.ru/cars/used/sale/bmw/x1/1105349715-0fac123d/',
         'https://auto.ru/cars/used/sale/bmw/x5/1105619374-41017faf/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105537819-ee520e3e/',
 

         'https://auto.ru/cars/used/sale/bmw/x7/1104885993-0369f7d8/',
         'https://auto.ru/cars/used/sale/bmw/7er/1105108821-82595f78/',
         'https://auto.ru/cars/used/sale/bmw/5er/1104959381-a2d0a2f6/',
         'https://auto.ru/cars/used/sale/bmw/x6/1105150845-fbbbdd61/',
         'https://auto.ru/cars/used/sale/bmw/7er/1103637231-3eb5d40e/',
         'https://auto.ru/cars/used/sale/bmw/7er/1104670462-957d1de3/',
         'https://auto.ru/cars/used/sale/bmw/x6_m/1105605911-1e30bebc/',
         'https://auto.ru/cars/used/sale/bmw/x6_m/1105604671-5503f036/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105604201-c9cecc3c/',
         'https://auto.ru/cars/used/sale/bmw/x4/1103878094-2a936544/',
         'https://auto.ru/cars/used/sale/bmw/x6/1105483643-fc3a6a76/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105601399-129e1fff/',
         'https://auto.ru/cars/used/sale/bmw/5er/1105444468-30dbd99a/',
         'https://auto.ru/cars/used/sale/bmw/3er/1105497097-11fa10

Процесс парсинга очень долгий и есть большая верятность что сервер откажет в доступе. Поэтому будем записывать промежуточные данные в файлы. Также для ускорения процеса будем параллельно отправлять сразу по 100 запросов используя библиотеку concurrent.futures

In [ ]:
with open('cars.json', 'r', encoding='utf-8') as f:
    #открыли файл с данными
    dict_cars = json.load(f)

# создадим список с интереусующими нас признаками
col = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
       'fuelType', 'image', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 'parsing_unixtime','priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'Владельцы', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня','price']

df=pd.DataFrame(columns=col) # создаем датафрейм куда будем записывать информацию
a=[0]

for brand in dict_cars.keys():
    links = dict_cars[brand]

    while a[-1]<len(links):
        a.append(a[-1]+100)

    a[-1]=len(links)

    for element in range(0,len(a)) :
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = []

            for i in range(a[element-1],a[element]):
                print (i)
                futures.append(executor.submit(fill_car_dict,(links[i])))

            for future in concurrent.futures.as_completed(futures):

                if future.result()!=None:
                    dt = pd.DataFrame(future.result())
                    dt=dt.reindex(columns=col)
                    df=pd.concat([df,dt],ignore_index=True)

        df.to_csv(str(brand)+str(element)+'.csv',index=False)  
        try:
            os.remove(str(brand)+str(element-1)+'.csv')
        except:
            print (str(brand)+str(element-1)+'.csv is not found') 

bmw-1.csv is not found
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
bmw0.csv is not found
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
bmw1.csv is not found
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-17-8ca3df9dedd2>", line 30, in <module>
    for future in concurrent.futures.as_completed(futures):
  File "C:\ProgramData\Anaconda3\lib\concurrent\futures\_base.py", line 244, in as_completed
    waiter.event.wait(wait_timeout)
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 558, in wait
    signaled = self._cond.wait(timeout)
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 302, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-8ca3df9dedd2>", line 35, in <module>
    df=pd.concat([df,dt],ignore_index=True)
  File "C:\ProgramData\Anaconda3\lib\concurrent\futures\_base.py", line 636, in __exit__
    self.shutdown(wait=True)
 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-17-8ca3df9dedd2>", line 30, in <module>
    for future in concurrent.futures.as_completed(futures):
  File "C:\ProgramData\Anaconda3\lib\concurrent\futures\_base.py", line 244, in as_completed
    waiter.event.wait(wait_timeout)
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 558, in wait
    signaled = self._cond.wait(timeout)
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 302, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-8ca3df9dedd2>", line 35, in <module>
    df=pd.concat([df,dt],ignore_index=True)
  File "C:\ProgramData\Anaconda3\lib\concurrent\futures\_base.py", line 636, in __exit__
    self.shutdown(wait=True)
 

In [ ]:
# Из сохраненных файлов удаляем индексы и все подгружаем
data=pd.read_csv('audi.csv')
d_bmw=pd.read_csv('bmw.csv')
d_honda=pd.read_csv('honda.csv')
d_infiniti=pd.read_csv('infiniti.csv')
d_lexus=pd.read_csv('lexus.csv')
d_mercedes=pd.read_csv('mercedes.csv')
d_mitsubishi=pd.read_csv('mitsubishi.csv')
d_nissan=pd.read_csv('nissan.csv')
d_skoda=pd.read_csv('skoda.csv')
d_toyota=pd.read_csv('toyota.csv')
d_volkswagen=pd.read_csv('volkswagen.csv')
d_volvo=pd.read_csv('volvo.csv')

In [ ]:
# Объединяем в один датасет
data=pd.concat([data, d_bmw,d_honda,d_infiniti,d_lexus,d_mercedes,d_mitsubishi,d_nissan,d_skoda,d_toyota,
             d_volkswagen,d_volvo],ignore_index=True)

In [ ]:
# Удаляем дубликаты
data.drop_duplicates(inplace=True)
data.isnull().sum()

In [ ]:
# Удаляем пропуски
data.dropna(subset=['price','Владельцы','ПТС','Состояние'],inplace=True)

In [ ]:
# Записываем готовый датасет
data.to_csv('data.csv',index=False)